# 台鐵車站的資料分析與視覺化 Part 3. 日期處理

這個part可說是上一部份的繼續。因為我還必須處理每日運量的資料，這牽涉到時間序列，處理上又會更加複雜，因此獨立成一個notebook。

## 1. 前處理：車站統一

匯入所需套件

In [1]:
%config Completer.use_jedi = False
%matplotlib inline
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import seaborn as sns

In [3]:
df_train = pd.read_csv('2020_Station.csv', parse_dates=[0])

看看資料，我對於各車站每日有多少人進或出站不是那麼有興趣，而是對於一日流通量較感興趣，因此我們要整理出這樣的欄位。

In [4]:
df_train.head()

,trnOpDate,staCode,gateInComingCnt,gateOutGoingCnt
0,2020-01-01,900,7552,8154
1,2020-01-01,910,1020,1135
2,2020-01-01,920,1623,1972
3,2020-01-01,930,4130,4813
4,2020-01-01,940,1818,2128


In [5]:
df_train['Total'] = df_train.gateInComingCnt + df_train.gateOutGoingCnt

讀入之前的整理好的資料集。目的是為了讓兩邊的車站統一，以免後續分析困難。

In [6]:
df_station = pd.read_csv('df_station.csv', index_col=0)

In [7]:
df_station.head()

,stationCode,stationName,stationEName,stationAddrTw,stationAddrEn,stationTel,gps,Level,Link,Open,Line
0,900,基隆,Keelung,基隆市中山區民治里1鄰中山一路16之 1號,"No. 16-1, Zhongshan 1st Rd, Zhongshan Dist., K...",02-24263743,25.13401 121.74013,一等,/wiki/%E5%9F%BA%E9%9A%86%E8%BB%8A%E7%AB%99,1891-10-20,縱貫線、基隆捷運（規劃中）
1,910,三坑,Sankeng,基隆市仁愛區德厚里龍安街 206 號,"No. 206, Long’an St., Ren`ai Dist., Keelung City",02-24230289,25.12305 121.74202,簡易,/wiki/%E4%B8%89%E5%9D%91%E8%BB%8A%E7%AB%99,2003-05-09,縱貫線
2,920,八堵,Badu,基隆市暖暖區八南里八堵路 142 號,"No. 142, Badu Rd., Nuannuan Dist., Keelung City",02-24560841,25.10843 121.72898,二等,/wiki/%E5%85%AB%E5%A0%B5%E8%BB%8A%E7%AB%99,1899-07-20,縱貫線、宜蘭線
3,930,七堵,Qidu,基隆市七堵區長興里東新街 2 號,"No. 2, Dongxin St., Qidu Dist., Keelung City",02-24553426,25.09294 121.71415,一等,/wiki/%E4%B8%83%E5%A0%B5%E8%BB%8A%E7%AB%99,1891-10-20,縱貫線
4,940,百福,Baifu,基隆市七堵區堵南里明德三路 1 之 1 號,"No. 1-1, Mingde 3rd Rd., Qidu Dist., Keelung City",02-24528372,25.07803 121.69373,簡易,/wiki/%E7%99%BE%E7%A6%8F%E8%BB%8A%E7%AB%99,2007-05-08,縱貫線


In [9]:
df_station.shape

(240, 11)

看看兩個資料集之間有沒有不共通的車站，發現只有一個：枋野。不過這個車站也還好，因為沒有提供客運服務，所以到時候合併時捨棄是可以接受的。

In [10]:
for i in df_station.stationCode:
    if i not in df_train.staCode.unique():
        print(df_station[df_station.stationCode == i]['stationName'])

161    枋野
Name: stationName, dtype: object


## 2. 日期缺值處理

接著是重頭戲！開始處理時間資料了。有些車站可能會因為當日沒有運量、資料缺失等，導致該車站紀錄筆數不足當年日數，必須先修正這個問題。

先來看看有多少車站存在這樣的狀況。

In [11]:
df_train.groupby('staCode')['Total'].size().value_counts()

366    226
360      2
345      1
356      1
357      1
358      1
362      1
363      1
364      1
365      1
126      1
249      1
251      1
Name: Total, dtype: int64

將不足一年的車站取出來，要將剩餘的天數補0放回去。

>在這邊假設沒有資料是因為沒有運量。

In [12]:
not366 = list(df_train.groupby('staCode')['Total'].size()[df_train.groupby('staCode')['Total'].size() != 366].index)

In [13]:
df_train.groupby('staCode')['Total'].size()[df_train.groupby('staCode')['Total'].size() != 366]

staCode
5130    249
5140    126
5160    251
6040    365
7170    345
7331    356
7332    364
7333    357
7334    362
7335    360
7336    363
7361    360
7362    358
Name: Total, dtype: int64

看看有哪些車站不足366天，會發現大部分確實都是小站，但有些觀光車站，例如十分、平溪等，卻也不足366天，有點不太合理。

但觀察一下，會發現整條平溪線都有這樣的狀況。我突然想到，2020年12月平溪線有發生自然災害導致列車停駛，如此一來，確實有可能會有幾天沒有運量而沒有紀錄。稍後再來看看是不是真的這樣。先把缺失資料補回去。

In [14]:
df_station[df_station.stationCode.isin(not366)][['stationCode','stationName','Line']]

,stationCode,stationName,Line
158,5130,加祿,南迴線
159,5140,內獅,南迴線、恆春線（規劃中）
160,5160,枋山,南迴線
172,6040,瑞和,臺東線
211,7170,中里,宜蘭線
228,7331,大華,平溪線
229,7332,十分,平溪線
230,7333,望古,平溪線
231,7334,嶺腳,平溪線
232,7335,平溪,平溪線


In [15]:
df_train_days = df_train.copy()

In [16]:
for i in not366:
    for j in pd.date_range('2020-01-01', periods=366):
        if j not in list(df_train_days[df_train_days.staCode == i].trnOpDate):
            df_train_days = df_train_days.append(pd.Series({'trnOpDate': j, 'staCode': i,
                                                            'gateInComingCnt': 0, 'gateOutGoingCnt': 0,
                                                            'Total': 0}), ignore_index=True)

In [17]:
df_train_days.groupby('staCode')['Total'].size().value_counts()

366    239
Name: Total, dtype: int64

In [18]:
df_train_days.shape

(87474, 5)

In [19]:
df_train_days = df_train_days.sort_values(by=['trnOpDate', 'staCode']).reset_index(drop=True)

In [20]:
df_train_days

,trnOpDate,staCode,gateInComingCnt,gateOutGoingCnt,Total
0,2020-01-01,900,7552,8154,15706
1,2020-01-01,910,1020,1135,2155
2,2020-01-01,920,1623,1972,3595
3,2020-01-01,930,4130,4813,8943
4,2020-01-01,940,1818,2128,3946
...,...,...,...,...,...
87469,2020-12-31,7360,3523,3602,7125
87470,2020-12-31,7361,3,18,21
87471,2020-12-31,7362,28,34,62
87472,2020-12-31,7380,609,604,1213


補完之後，來看看我的假設是否正確。先從平溪(7335)開始看起，果然12月有一陣子都沒有運量，剛好就是意外發生的時間點。Bingo!

In [21]:
df_train_days[(df_train_days.Total == 0) & (df_train_days.staCode == 7335)]

,trnOpDate,staCode,gateInComingCnt,gateOutGoingCnt,Total
81252,2020-12-05,7335,0,0,0
81491,2020-12-06,7335,0,0,0
82208,2020-12-09,7335,0,0,0
82447,2020-12-10,7335,0,0,0
82686,2020-12-11,7335,0,0,0
82925,2020-12-12,7335,0,0,0


In [22]:
df_train_days[(df_train_days.staCode == 7335) & (df_train_days.trnOpDate.isin(pd.date_range('2020-12-01', '2020-12-15')))]

,trnOpDate,staCode,gateInComingCnt,gateOutGoingCnt,Total
80296,2020-12-01,7335,35,59,94
80535,2020-12-02,7335,91,63,154
80774,2020-12-03,7335,66,304,370
81013,2020-12-04,7335,10,28,38
81252,2020-12-05,7335,0,0,0
81491,2020-12-06,7335,0,0,0
81730,2020-12-07,7335,0,7,7
81969,2020-12-08,7335,0,1,1
82208,2020-12-09,7335,0,0,0
82447,2020-12-10,7335,0,0,0


## 3. 建立資料樣態

為了方便之後使用，我決定將一部分的運量資料整理到車站資料中。

In [23]:
passenger_info = df_train_days.groupby('staCode').Total.agg(['max', 'min', 'mean', 'sum'])

In [24]:
df_stapass = df_station.merge(passenger_info, left_on='stationCode', right_on='staCode')

In [25]:
df_stapass_sim = df_stapass[['stationCode', 'stationName', 'Level', 'Open', 'Line', 'max', 'min', 'mean', 'sum']]

In [26]:
df_stapass_sim

,stationCode,stationName,Level,Open,Line,max,min,mean,sum
0,900,基隆,一等,1891-10-20,縱貫線、基隆捷運（規劃中）,20685,6897,13492.571038,4938281
1,910,三坑,簡易,2003-05-09,縱貫線,3202,1417,2522.655738,923292
2,920,八堵,二等,1899-07-20,縱貫線、宜蘭線,5952,2152,4175.448087,1528214
3,930,七堵,一等,1891-10-20,縱貫線,15654,5223,11144.581967,4078917
4,940,百福,簡易,2007-05-08,縱貫線,6675,2344,4857.756831,1777939
...,...,...,...,...,...,...,...,...,...
234,7360,瑞芳,一等,1919-05-05,宜蘭線、深澳線,15648,3761,7670.453552,2807386
235,7361,海科館,招呼,2014-01-09,深澳線,1139,0,168.267760,61586
236,7362,八斗子,招呼,1967-08-25,深澳線,1894,0,343.049180,125556
237,7380,四腳亭,三等,1919-05-05,宜蘭線,1857,561,1149.442623,420696


In [27]:
df_stapass_sim.groupby('Level')['sum'].agg(['mean', 'std', 'sum'])

,mean,std,sum
Level,,,
一等,6.985579e+06,5.165638e+06,195596225
三等,8.124858e+05,1.115565e+06,54436551
丙簡,2.435785e+05,2.330589e+05,487157
乙簡,4.071473e+05,6.692661e+05,7328651
二等,1.760619e+06,1.798334e+06,44015470
招呼,8.460207e+04,9.176550e+04,3384083
特等,1.839965e+07,1.393922e+07,73598606
甲簡,2.365472e+05,1.707416e+05,2838566
簡易,8.110398e+05,1.035434e+06,34874713


同時我也決定將一部分的車站資料放到運量資料中，也許之後會用的到

In [28]:
df_tr_count = df_train_days.merge(df_station[['stationCode', 'stationName', 'Level']],
                                  left_on='staCode', right_on='stationCode').drop('stationCode', axis=1).set_index('trnOpDate')

In [29]:
df_tr_count

,staCode,gateInComingCnt,gateOutGoingCnt,Total,stationName,Level
trnOpDate,,,,,,
2020-01-01,900,7552,8154,15706,基隆,一等
2020-01-02,900,7709,7240,14949,基隆,一等
2020-01-03,900,8713,8086,16799,基隆,一等
2020-01-04,900,8625,8407,17032,基隆,一等
2020-01-05,900,8015,7355,15370,基隆,一等
...,...,...,...,...,...,...
2020-12-27,7390,302,378,680,暖暖,招呼
2020-12-28,7390,419,477,896,暖暖,招呼
2020-12-29,7390,410,475,885,暖暖,招呼


In [45]:
df_tr_count.to_csv('train_count.csv')

In [46]:
df_stapass_sim.to_csv('station_passenger_simple_summary.csv')